In [2]:
import base64
import hmac
import json
import time
import requests
import toml

from urllib.parse import urljoin

# params
base_url = 'https://www.dojqutmbiuuo.com'
api_config = toml.load('../config/api.toml')
api_key = api_config['api_key']
secret_key = api_config['secret_key']
passphrase = api_config['passphrase']

In [3]:
def request(method, uri, params=None, body=None, headers=None, auth=False):
    """Initiate network request
   @param method: request method, GET / POST / DELETE / PUT
   @param uri: request uri
   @param params: dict, request query params
   @param body: dict, request body
   @param headers: request http header
   @param auth: boolean, add permission verification or not
   """
    if params:
        query = "&".join(
            ["{}={}".format(k, params[k]) for k in sorted(params.keys())]
        )
        uri += "?" + query
    url = urljoin(base_url, uri)

    if auth:
        timestamp = (
                str(time.time()).split(".")[0]
                + "."
                + str(time.time()).split(".")[1][:3]
        )
        if body:
            body = json.dumps(body)
        else:
            body = ""
        message = str(timestamp) + str.upper(method) + uri + str(body)
        mac = hmac.new(
            bytes(secret_key, encoding="utf8"),
            bytes(message, encoding="utf-8"),
            digestmod="sha256",
        )
        d = mac.digest()
        sign = base64.b64encode(d)

        if not headers:
            headers = {}
        headers["Content-Type"] = "application/json"
        headers["OK-ACCESS-KEY"] = api_key
        headers["OK-ACCESS-SIGN"] = sign
        headers["OK-ACCESS-TIMESTAMP"] = str(timestamp)
        headers["OK-ACCESS-PASSPHRASE"] = passphrase
    result = requests.request(
        method, url, data=body, headers=headers, timeout=10
    ).json()
    if result.get("code") and result.get("code") != "0":
        return None, result
    return result, None

In [4]:
from datetime import datetime

def str_to_timestamp_ms(date_str, format_str="%Y-%m-%d"):
    # 将字符串转换为datetime对象
    dt = datetime.strptime(date_str, format_str)
    # 获取时间戳（秒）并转换为毫秒
    timestamp_ms = int(dt.timestamp() * 1000)
    return timestamp_ms

def timestamp_ms_to_str(timestamp_ms, format_str="%Y-%m-%d"):
    # 将毫秒转换为秒
    timestamp_s = timestamp_ms / 1000
    # 转换为datetime对象
    dt = datetime.fromtimestamp(timestamp_s)
    # 转换为指定格式的字符串
    return dt.strftime(format_str)

In [5]:
timestamp_str = "2024-11-09"
timestamp = str_to_timestamp_ms(timestamp_str, '%Y-%m-%d')
timestamp_ms_to_str(1730995200000)

'2024-11-08'

In [6]:
import pandas as pd
import time
from datetime import datetime, timedelta

def get_spot_klines(spot, begin, end, period='1D', limit=100):
    all_klines = []
    begin_dt = datetime.strptime(begin, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    
    while end_dt >= begin_dt:
        print(begin_dt)
        diff = abs((end_dt - begin_dt).days)
        params = {
            "instId": spot,
            "before": str_to_timestamp_ms(begin_dt.strftime("%Y-%m-%d")),
            "after": str_to_timestamp_ms((begin_dt + timedelta(days=min(diff, limit))).strftime("%Y-%m-%d")),
            "bar": period,
            "limit": limit,
        }
        
        try:
            result = request("GET", "/api/v5/market/history-candles", params=params, auth=True)
            klines = result[0]['data']
            all_klines.extend(klines)
            
            begin_dt += timedelta(days=limit-1)
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    # 处理所有收集到的数据
    df = None
    try:
        if all_klines:
            cols = ['timestamp', 'open', 'high', 'low', 'close', 'vol', 'volCcy', 'volCcyQuote', 'confirm']
            kline_dict = {col: [] for col in cols}
            
            for kline in all_klines:
                kline[0] = timestamp_ms_to_str(int(kline[0]))
                for idx, col in enumerate(cols):
                    kline_dict[col].append(kline[idx] if col == 'timestamp' else float(kline[idx]))
            
            df = pd.DataFrame(kline_dict)
            # 按时间排序并去重
            df = df.sort_values('timestamp').drop_duplicates(subset=['timestamp']).reset_index(drop=True)
            
    except Exception as e:
        print(f"Error processing data: {e}")
    
    return df

# 使用示例
df = get_spot_klines('BTC-USDT', '2019-01-01', '2024-12-21')
df
df.to_csv('./assets/btc.csv', index=False)

2019-01-01 00:00:00
2019-04-10 00:00:00
2019-07-18 00:00:00
2019-10-25 00:00:00
2020-02-01 00:00:00
2020-05-10 00:00:00
2020-08-17 00:00:00
2020-11-24 00:00:00
2021-03-03 00:00:00
2021-06-10 00:00:00
2021-09-17 00:00:00
2021-12-25 00:00:00
2022-04-03 00:00:00
2022-07-11 00:00:00
2022-10-18 00:00:00
2023-01-25 00:00:00
2023-05-04 00:00:00
2023-08-11 00:00:00
2023-11-18 00:00:00
2024-02-25 00:00:00
2024-06-03 00:00:00
2024-09-10 00:00:00
2024-12-18 00:00:00
